In [1]:
import nest_asyncio
nest_asyncio.apply()

# from utils import *

import chromadb

In [2]:
# pip install requests pdfminer.six

In [3]:
import asyncio
from typing import Dict, List, Optional, Union, Callable
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.formatting_utils import colored
from typing_extensions import Annotated
import autogen

from teachability import Teachability
from concurrent.futures import ThreadPoolExecutor, as_completed

import arxiv

db_dir = './teachability_db-3'
# check if db_dir exists, delete it if it does
import os
import shutil
if os.path.exists(db_dir): shutil.rmtree(db_dir)

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4-32k", "gpt-4", "gpt4", "gpt-35-turbo-16k", "gpt-4-0613", "gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613"]
    },
)



print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

from typing import Annotated
import concurrent.futures
# Assuming other necessary imports like `autogen`, `Teachability`, etc., are done earlier.

def initiate_chat_with_paper_info(paper, teachable_agent, user, query_text):
    user.initiate_chat(teachable_agent,
                       silent=True,
                       message=f"The following article is one of the articles that I found for '{query_text}' topic: \n\n '{paper.title}' by {paper.authors} updated on {paper.updated}: {paper.pdf_url} \nsummary: {paper.summary} \n?")

def process_query(query_text, n_results, teachable_agent, user):
    """Function to process each query and initiate chats for each paper found."""
    sort_by = arxiv.SortCriterion.Relevance
    papers = arxiv.Search(query=query_text, max_results=n_results, sort_by=sort_by)
    
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(initiate_chat_with_paper_info, paper, teachable_agent, user, query_text) for paper in arxiv.Client().results(papers)]
        for future in as_completed(futures):
            future.result()

def arxiv_retriever(queries: Annotated[List[str], "The list of query texts to search for."], 
                    n_results: Annotated[int, "The number of results to retrieve for each query."] = 10,
                    ) -> str:
    
    # Start by instantiating any agent that inherits from ConversableAgent.
    teachable_agent = autogen.ConversableAgent(
        name="teachable_agent",  # The name is flexible, but should not contain spaces to work in group chat.
        llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
    )

    # Instantiate the Teachability capability. Its parameters are all optional.
    teachability = Teachability(
        verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
        reset_db=False,  
        path_to_db_dir=db_dir,
        recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
    )

    # Now add the Teachability capability to the agent.
    teachability.add_to_agent(teachable_agent)

    user = autogen.UserProxyAgent(
        name="user",
        human_input_mode="NEVER",
        is_termination_msg=lambda x: "TERMINATE" in x.get("content"),
        max_consecutive_auto_reply=0,
        code_execution_config={"use_docker": False},
    )

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_query, query_text, n_results, teachable_agent, user) for query_text in queries]
        for future in as_completed(futures):
            future.result()

    # Instantiate a UserProxyAgent to represent the user. But in this notebook, all user input will be simulated.
    return f"Dear Researcher, Database updated with on the following topics: {', '.join([query for query in queries])}. Please go ahead with your task. Contact me for updates if needed. Your research Coordinator"


message = ["Overview of time series forecasting methods", "Deep learning for time series forecasting"]
# arxiv_retriever(message, n_results=3)

LLM models:  ['gpt-4', 'gpt-4-32k', 'gpt-4-0613', 'gpt-35-turbo', 'gpt-35-turbo-16k']


In [4]:
def create_teachable_groupchat(assitant_name, user_name, db_dir, config_list, verbosity=0):
    
    # Start by instantiating any agent that inherits from ConversableAgent.
    assistant = autogen.ConversableAgent(
        name=assitant_name,  # The name is flexible, but should not contain spaces to work in group chat.
        llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
    )

    # Instantiate the Teachability capability. Its parameters are all optional.
    teachability = Teachability(
        verbosity=verbosity,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
        reset_db=False,  
        path_to_db_dir=db_dir,
        recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
    )

    # Now add the Teachability capability to the agent.
    teachability.add_to_agent(assistant)

    user = autogen.UserProxyAgent(
        name=user_name,
        human_input_mode="NEVER",
        is_termination_msg=lambda x: "TERMINATE" in x.get("content"),
        max_consecutive_auto_reply=0,
        code_execution_config={"use_docker": False},
    )

    return assistant, user

In [5]:
prompt = "For each memorization task, initiate your process with 'MEMORIZE_ARTICLE: The following passage is extracted from an article, titled article_title [article_url]: \n\n' Delve into the passage to discern and assess its key insights. If the content presents noteworthy information, make a point to memorize these details. Conversely, if the passage does not offer significant insights, there's no need to commit it to memory. Upon choosing to memorize, finalize your notes by including both the article's title and its URL, employing the format '[source: article_title, article_url]' for efficient future access and verification."

instract_assistant, instract_user = create_teachable_groupchat("instract_assistant", "instract_user", db_dir, config_list, verbosity=3)

instract_user.initiate_chat(instract_assistant, silent=False, message=prompt)

instract_user (to instract_assistant):

For each memorization task, initiate your process with 'MEMORIZE_ARTICLE: The following passage is extracted from an article, titled article_title [article_url]: 

' Delve into the passage to discern and assess its key insights. If the content presents noteworthy information, make a point to memorize these details. Conversely, if the passage does not offer significant insights, there's no need to commit it to memory. Upon choosing to memorize, finalize your notes by including both the article's title and its URL, employing the format '[source: article_title, article_url]' for efficient future access and verification.

--------------------------------------------------------------------------------
instract_assistant (to analyzer):

For each memorization task, initiate your process with 'MEMORIZE_ARTICLE: The following passage is extracted from an article, titled article_title [article_url]: 

' Delve into the passage to discern and assess its key

ChatResult(chat_id=None, chat_history=[{'content': "For each memorization task, initiate your process with 'MEMORIZE_ARTICLE: The following passage is extracted from an article, titled article_title [article_url]: \n\n' Delve into the passage to discern and assess its key insights. If the content presents noteworthy information, make a point to memorize these details. Conversely, if the passage does not offer significant insights, there's no need to commit it to memory. Upon choosing to memorize, finalize your notes by including both the article's title and its URL, employing the format '[source: article_title, article_url]' for efficient future access and verification.", 'role': 'assistant'}, {'content': 'Understood. Please go ahead and provide the passage for memorization along with the article title and URL, and I will follow your instructions to process the information accordingly.', 'role': 'user'}], summary='Understood. Please go ahead and provide the passage for memorization alo

In [6]:
import requests
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.pdf import partition_pdf

import os

import requests
import pdfplumber
import os
from typing import Optional

def download_pdf(url, save_path):
    """Download a PDF from a given URL."""
    response = requests.get(url)
    with open(save_path, 'wb') as f:
        f.write(response.content)

def initiate_chat_read_paper(text, teachable_agent, user, article):
    user.initiate_chat(teachable_agent,
                       silent=False,
                       message=f"MEMORIZE_ARTICLE: The following passage is extracted from an article titled '{article}': \n\n {text}."
                    )


def get_pdf(url: Annotated[str, "The URL of the PDF to convert."]) -> str:

    paper_reader, reader_user = create_teachable_groupchat("paper_reader", "reader_user", db_dir, config_list, verbosity=0)
    
    output_dir = './pdf_output'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    paper_id = url.split('/')[-1].replace('.pdf', '')
    search_by_id = arxiv.Search(id_list=[paper_id])
    paper = list(arxiv.Client().results(search_by_id))[0]
    title = paper.title
    
    pdf_filename = url.split('/')[-1]
    pdf_path = os.path.join(output_dir, pdf_filename)
    
    download_pdf(url, pdf_path)
    elements = partition_pdf(filename=pdf_path)
    chunked_elements = chunk_by_title(elements)

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(initiate_chat_read_paper, text, paper_reader, reader_user, f"{title} [{url}]") for text in chunked_elements]
        for future in as_completed(futures):
            future.result()
    
    return f"The article {title} [{url}] has been read and the content has been shared with you in your memory."

# Example usage
pdf_url = "https://arxiv.org/ftp/arxiv/papers/2305/2305.17873.pdf"
get_pdf(pdf_url)


********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
    Location = ./teachability_db-3/uid_text_dict.pkl
reader_user (to paper_reader):

reader_user (to paper_reader):

MEMORIZE_ARTICLE: The following passage is extracted from an article titled 'The Digital Divide in Process Safety: Quantitative Risk Analysis of Human-AI Collaboration [https://arxiv.org/ftp/arxiv/papers/2305/2305.17873.pdf]': 

 Digital technologies have dramatically accelerated the digital transformation in process industries, boosted new industrial applications, upgraded the production system, and enhanced operational efficiency. In contrast, the challenges and gaps between human and artificial intelligence (AI) have become more and more prominent, whereas the digital divide in process safety is aggregating. The study attempts to address the following questions: (i)What is AI in the process safety context? (ii)What is.
reader_user (to paper_reader):

MEMORIZE_ARTICLE: Th

Insert of existing embedding ID: 8
Add of existing embedding ID: 8



>>>>>>>> USING AUTO REPLY...
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
paper_reader (to reader_user):

The information has been memorized successfully. If you wish to access or inquire about the contents of the article titled 'The Digital Divide in Process Safety: Quantitative Risk Analysis of Human-AI Collaboration', or any related subject, feel free to ask, and I will provide the stored information. [source: The Digital Divide in Process Safety: Quantitative Risk Analysis of Human-AI Collaboration, https://arxiv.org/ftp/arxiv/papers/2305/2305.17873.pdf]

--------------------------------------------------------------------------------
reader_user (to paper_reader):

MEMORIZE_ARTICLE: The following passage is extracted from an article titled 'The Digital Divide in Process Safety: Quantitative Risk Analysis of Human-AI Collaboration [https://arxiv.org/ftp/arxiv/papers/23

Insert of existing embedding ID: 23
Add of existing embedding ID: 23



>>>>>>>> USING AUTO REPLY...
paper_reader (to reader_user):

The passage and the additional details you provided from 'The Digital Divide in Process Safety: Quantitative Risk Analysis of Human-AI Collaboration' have been memorized. If you have specific questions or need information on this topic, feel free to ask!

--------------------------------------------------------------------------------
reader_user (to paper_reader):

MEMORIZE_ARTICLE: The following passage is extracted from an article titled 'The Digital Divide in Process Safety: Quantitative Risk Analysis of Human-AI Collaboration [https://arxiv.org/ftp/arxiv/papers/2305/2305.17873.pdf]': 

 2.2.3. IDHEAS

IDHEAS-Data is the most advanced HRA method, providing specific human error probability simulated by SACADA. IDHEAS-Data (Table 2) contains 4 contexts with 20 performance-influencing factors (PIFs), 133 attributes, and 5 actions with 28 sub- actions. Combining the attributes and sub-actions, it consists of 555 combinations

...

Insert of existing embedding ID: 48
Add of existing embedding ID: 48



>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
paper_reader (to reader_user):

Article content stored:

- Title of article: "The Digital Divide in Process Safety: Quantitative Risk Analysis of Human-AI Collaboration."
- Link of article: [https://arxiv.org/ftp/arxiv/papers/2305/2305.17873.pdf]
- Points to remember:
  - Significant process accidents haven't been reduced as per Khan et al., 2021.
  - Continuous improvement in automation and digitization can't entirely eliminate risk.
  - Research on AI reliability has evolved from components to system application level focusing on SIS and SIL.
  - AI's definition according to ISO/IEC standards involves acquiring, processing, and applying knowledge and skills.
  - Human error's importance was highlighted after the Three Mile Island accident and WASH-1400 Reactor Safety Study.
  - Concerns over dealing with data uncertainty in AI failure and human error probabilities.
  - A model is needed to illustrate and quantify the risk in

Insert of existing embedding ID: 91
Add of existing embedding ID: 91



>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
paper_reader (to reader_user):

I have memorized the details of the article as requested. You can now ask me questions or seek clarifications about the content or concepts from the passages, and I'll be able to provide information based on what I've stored. Just let me know how I can assist you!

--------------------------------------------------------------------------------
reader_user (to paper_reader):

MEMORIZE_ARTICLE: The following passage is extracted from an article titled 'The Digital Divide in Process Safety: Quantitative Risk Analysis of Human-AI Collaboration [https://arxiv.org/ftp/arxiv/papers/2305/2305.17873.pdf]': 

 Kirwan, B. (1998). Human error identification techniques for risk assessment of high

risk systems - Part 1: Review and evaluation of techniques. In Applied Ergonomics (Vol. 29, Issue 3, pp. 157–177). https://doi.org/10.1016/S0003- 6870(98)00010-6

Lee, J., Cameron, I.

'The article The Digital Divide in Process Safety: Quantitative Risk Analysis of Human-AI Collaboration [https://arxiv.org/ftp/arxiv/papers/2305/2305.17873.pdf] has been read and the content has been shared with you in your memory.'

In [7]:
# Configuration for the Language Model (LLM)
llm_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 120,
    "seed": 42,
}

# Configuration for the manager using the same config_list as llm_config
manager_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 60,
    "seed": 42,
}

In [8]:
# System message constants for different roles
PI_prompt = """You are a Principal investigator (PI): You are the leader of the research team who asks the questions and gives task.
You MUST make sure that the research team is on the right track and the research is going in the right direction. 
You should check the work of the researcher, coordinator and Planner and provide feedback to improve the quality of the work or confirm the work.
reply 'TERMINATE' in the end when everything is done.
"""


COORDINATOR = """You are a Research coordinator: This is the person who coordinates the various aspects of the research project. 
you are equipped wih a tool that could help you to query for the arxiv api. 
You MUST rephrase research questions into a list of queries (at least 5) for the arxiv api that cover the key aspects of the research questions. 
"""

RESEARCHER = """You are a Researcher: This is the person who performs the research and writes the final report/article.
You MUST take to account the feedback from the critic to improve the quality of the work.
The final report/article should include references to the sources you used to perform the research and also for reader to refer for more information.
The references MUST be in the format of the arxiv url.
You are equipped with a memory tool that could help you to retrieve the information you need. 
You are equipped with a tool that could help you to get the content of the pdf file from the url. In case you have a useful pdf url in your memory, which you can use to get the content of the pdf file, you should use the tool to get the content of the pdf file.
You MUST verify the information you retrieve from the memory tool before using it in your work an make sure it is accurate and enough to answer the question.
If there isn't enough information for you to perform your task, you should asked the PI to provide you with the missing information from Arxiv. You should mention what is missing in your request start you sentence exactly with "ARXIV REQUEST:". 
You should not generate answers that don't use the sources provided in the context.
"""
SUB1 = """You are a Sub-investigator (Sub-I): This is the assistant to the PI, who helps with the tasks of the PI with a step wise research plan with sub-research topics.
you MUST help PI to ensure that the research team is on the right track and the research is going in the right direction. 
In your role, you have the autonomy to question the provided content or the process presented in this group chat and can request corrections or seek clarification if there is something that appears to be missing or unclear after executing a given task. If at any point you find yourself confused or in need of assistance, do not hesitate to reach out to the group chat manager, who can guide you or delegate the task to another qualified participant.
Reply 'TERMINATE' in the end when everything is done.
"""

# If there isn't enough information below, you should reply exactly 'UPDATE CONTEXT'.
QNA_PROMPT = """Assistant helps the researchers with searching information from the arxiv API. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. Do not generate answers that don't use the sources below. 
In your role, you have the autonomy to question the provided content or the process presented in this group chat and can request corrections or seek clarification if there is something that appears to be missing or unclear after executing a given task. If at any point you find yourself confused or in need of assistance, do not hesitate to reach out to the group chat manager, who can guide you or delegate the task to another qualified participant.
For tabular information return it as an html table. Do not return markdown format. If the question is not in English, answer in the language used in the question.
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brackets to reference the source, for example [info1.txt]. Don't combine sources, list each source separately.
User's question is: {input_question}

Context is: {input_context}"""

PLANNER = """Planner. Suggest a plan. Revise the plan based on feedback from PI and critic, until PI approval.
The plan may involve a Research Coordinator to to rephrase research questions into key word queries for the arxiv api and a Researcher who could performs the research of paper's summaries. 
Explain the plan first. Be clear which step is performed by an Research Coordinator, and which step is performed by a Researcher.
"""

# Termination message definition
termination_msg = (
    lambda x: isinstance(x, dict)
    and str(x.get("content", "")).upper() == "TERMINATE"
)

# Agent definitions
principalInvestigator = autogen.UserProxyAgent(
    name="PI",
    is_termination_msg=termination_msg,
    human_input_mode="TERMINATE",
    system_message=PI_prompt,
    llm_config=llm_config, 
    code_execution_config=False,
    description="Principal investigator (PI) is the leader of the research team who asks the questions and gives task."
)


planner = autogen.AssistantAgent(
    name="Planner",
    system_message=PLANNER,
    llm_config=llm_config,
    description="Planner suggests a plan and revises the plan based on feedback from PI and critic, until PI approval."
)

subInvestigator = autogen.AssistantAgent(
    name="Sub-I",
    is_termination_msg=termination_msg,
    system_message=SUB1,
    llm_config=llm_config,
    description="Sub-investigator (Sub-I) is the assistant to the PI, who helps with the tasks of the PI with a step wise research plan with sub-research topics."
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check the work of researcher, research coordinator and Planner and provide feedback to improve the quality of the work",
    llm_config=llm_config,
    description="Critic is responsible for double checking the work of researcher, research coordinator and Planner and provide feedback to improve the quality of the work"
)


researchCoordinator = autogen.AssistantAgent(
    name="ResearchCoordinator",
    is_termination_msg=termination_msg,
    system_message=COORDINATOR,  # COORDINATOR should be a predefined string variable
    llm_config=llm_config,
    description="Research coordinator is the person who rephrase research questions into key word queries for the arxiv api."
)

# create a UserProxyAgent instance named "user_proxy"
RC_proxy = autogen.UserProxyAgent(
    name="ResearchCoordinator_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "content" in x
    and x["content"] is not None
    and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "ResearchCoordinator",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    description="assist Research coordinator to query for the arxiv api."
)

researcher = autogen.AssistantAgent(
    name="Researcher",
    is_termination_msg=termination_msg,
    system_message=RESEARCHER, 
    llm_config=llm_config,
    description="Researcher is the person who performs the research of paper's summaries."
)


from chromadb.config import Settings

# db_dir = './teachability_db_AI_safety'
# settings = Settings(
#             anonymized_telemetry=False, allow_reset=True, is_persistent=True, persist_directory=db_dir
#         )

# Researcher_aid = RetrieveUserProxyAgent(
#     name="RAG_USER",
#     is_termination_msg=termination_msg,
#     human_input_mode="NEVER",
#     max_consecutive_auto_reply=3,
#     llm_config=llm_config, 
#     retrieve_config={
#         "task": "code",
#         "customized_prompt": QNA_PROMPT, 
#         "docs_path": None,
#         "model": config_list[0]["model"],
#         "client": chromadb.Client(settings),
#         "collection_name": "memos",
#         "get_or_create": True,
#     },
#     code_execution_config=False,  # we don't want to execute code in this case.
#     description="Assistant has extra content retrieval power and can provide team members with context in database.",
# )

# def chromadb_retriever(
#     message: Annotated[
#         str,
#         "Refined message which keeps the original meaning and can be used to retrieve content for code generation and question answering.",
#     ],
#     n_results: Annotated[int, "number of results. it should be at least 10."],
# ) -> str:
#     Researcher_aid.n_results = n_results  # Set the number of results to be retrieved.
#     # Check if we need to update the context.
#     update_context_case1, update_context_case2 = Researcher_aid._check_update_context(message)
#     if (update_context_case1 or update_context_case2) and Researcher_aid.update_context:
#         Researcher_aid.problem = message if not hasattr(Researcher_aid, "problem") else Researcher_aid.problem
#         _, ret_msg = Researcher_aid._generate_retrieve_user_reply(message)
#     else:
#         _context = {"problem": message, "n_results": n_results}
#         ret_msg = Researcher_aid.message_generator(Researcher_aid, None, _context)
#     return ret_msg if ret_msg else message

# message = "Overview of time series forecasting methods"
# chromadb_retriever(message, n_results=3)

# Instantiate the Teachability capability. Its parameters are all optional.
teachability = Teachability(
    verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=False,
    path_to_db_dir=db_dir,
    recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
)

# Now add the Teachability capability to the agent.
teachability.add_to_agent(researcher)


********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
    Location = ./teachability_db-3/uid_text_dict.pkl


In [9]:
from autogen import Agent
from typing import List, Dict

def custom_speaker_selection_func(last_speaker: Agent, groupchat: autogen.GroupChat):
    """Define a customized speaker selection function.
    A recommended way is to define a transition for each speaker in the groupchat.
    # principalInvestigator, planner, researchCoordinator, researcher, critic, RC_proxy, Researcher_aid

    Returns:
        Return an `Agent` class or a string from ['auto', 'manual', 'random', 'round_robin'] to select a default method to use.
    """
    messages = groupchat.messages

    if len(messages) <= 1:
        # first, let the researchCoordinator retrieve relevant data populate db
        return researchCoordinator

    # if last_speaker is researchCoordinator:
    #     # if the last message is from researchCoordinator, let the planner to speak
    #     return planner
    
    if last_speaker is planner:
        # if the last message is from researchCoordinator, let the planner to speak
        return critic

    # if last_speaker is RC_proxy:
    #     if messages[-1]["content"].strip() != "":
    #         # If the last message is from user and is not empty, let the writer to continue
    #         return researcher

    # elif last_speaker is engineer:
    #     if "```python" in messages[-1]["content"]:
    #         # If the last message is a python code block, let the executor to speak
    #         return executor
    #     else:
    #         # Otherwise, let the engineer to continue
    #         return engineer

    # elif last_speaker is executor:
    #     if "exitcode: 1" in messages[-1]["content"]:
    #         # If the last message indicates an error, let the engineer to improve the code
    #         return engineer
    #     else:
    #         # Otherwise, let the writer to speak
    #         return writer

    # elif last_speaker is researcher and messages[-1] does not have 'tool_calls' in it:
    # elif last_speaker is researcher and "tool_calls" not in messages[-1]:
    #     # Always let the user to speak after the writer
    #     return critic

    else:
        # default to auto speaker selection method
        return "auto"



In [10]:
def _reset_agents():
    principalInvestigator.reset()
    subInvestigator.reset()
    researchCoordinator.reset()
    researcher.reset()
    planner.reset()
    critic.reset()
    RC_proxy.reset()
    # Researcher_aid.reset()


def call_chat(PROBLEM):
    _reset_agents()  # Resets the state of all the agents before starting the chat

    autogen.agentchat.register_function(
        arxiv_retriever,
        caller=researchCoordinator,
        executor=RC_proxy,
        name="arxiv_retriever",
        description="Retrieve content for question answering from arxiv."
    )

    autogen.agentchat.register_function(
        get_pdf,
        caller=researcher,
        executor=RC_proxy,
        name="get_pdf",
        description="retrieve the content of the pdf file from the url."
    )

    # autogen.agentchat.register_function(
    #     chromadb_retriever,
    #     caller=researcher,
    #     executor=RC_proxy,
    #     name="chromadb_retriever",
    #     description="Retrieve content for question answering from database."
    # )

    # Create the GroupChat manager instance.
    groupchat = autogen.GroupChat(
        agents=[principalInvestigator, planner, researchCoordinator, researcher, critic, RC_proxy],
        # agents=[principalInvestigator, planner, researcher, critic, Researcher_aid],
        messages=[],
        max_round=35,
        speaker_selection_method=custom_speaker_selection_func,
        allow_repeat_speaker=False,
    )

    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config=manager_config,
    )

    # Initialize the chat with the primary investigator as the proxy agent.
    principalInvestigator.initiate_chat(
        manager,
        message=PROBLEM
    )

    # Start chatting with the primary investigator acting as the user proxy agent.
    return principalInvestigator.chat_messages

# Example usage:
# You MUST Collect enough information from the arxiv API and summerize the information in the blog post.
PROBLEM = """Write blog post about the modelling of reliability and safety mechanisms in AI system. 
The focus MUST be on Large Language Models.
The blog post MUST be written in a way that is easy to understand for a non-technical audience.
The blog post MUST be up to date and include the latest research in the field in your database.
"""
messages = call_chat(PROBLEM)
print(messages)


PI (to chat_manager):

Write blog post about the modelling of reliability and safety mechanisms in AI system. 
The focus MUST be on Large Language Models.
The blog post MUST be written in a way that is easy to understand for a non-technical audience.
The blog post MUST be up to date and include the latest research in the field in your database.


--------------------------------------------------------------------------------
ResearchCoordinator (to chat_manager):

***** Suggested tool Call (call_wxWgttjc7zh7a2t5ibwnvqC5): arxiv_retriever *****
Arguments: 
{
  "queries": [
    "reliability of large language models",
    "safety mechanisms in AI for large language models",
    "best practices in AI safety and reliability",
    "ethical considerations in AI with focus on large language models",
    "recent advancements in AI reliability and safety"
  ],
  "n_results": 5
}
********************************************************************************

-----------------------------------

Insert of existing embedding ID: 117
Add of existing embedding ID: 117



>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...


Insert of existing embedding ID: 127
Add of existing embedding ID: 127



>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
ResearchCoordinator_proxy (to chat_manager):

ResearchCoordinator_proxy (to chat_manager):

***** Response from calling tool "call_wxWgttjc7zh7a2t5ibwnvqC5" *****
Dear Researcher, Database updated with on the following topics: reliability of large language models, safety mechanisms in AI for large language models, best practices in AI safety and reliability, ethical considerations in AI with focus on large language models, recent advancements in AI reliability and safety. Please go ahead with your task. Contact me for updates if needed. Your research Coordinator
**********************************************************************

--------------------------------------------------------------------------------
********************************
* SYNC DISK MEMORY with VEC_DB *
********************************
Researcher (to chat_manager):

Thank you for the update. I'

...

Insert of existing embedding ID: 1221
Add of existing embedding ID: 1221



>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
paper_reader (to reader_user):

Article memorized successfully. If you need any information from it or wish to discuss the concepts it entails, feel free to ask!

--------------------------------------------------------------------------------
reader_user (to paper_reader):

MEMORIZE_ARTICLE: The following passage is extracted from an article titled 'GraphextQA: A Benchmark for Evaluating Graph-Enhanced Large Language Models [http://arxiv.org/pdf/2310.08487v1]': 

 There have also been attempts to integrate knowl- edge graph embeddings into language generation. For example, (Zhou et al., 2018) retrieves rele- vant knowledge subgraphs based on user posts and generates responses by attentively reading the retrieved knowledge graphs. ConceptFlow (Zhang et al., 2020) incorporates graph embeddings through graph neural networks (GNNs) into con- text representation and utilizes them to predict the.

----------------------------------

Insert of existing embedding ID: 1245
Add of existing embedding ID: 1245



>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
paper_reader (to reader_user):

The passage has been memorized. If you need information about the 'GraphextQA: A Benchmark for Evaluating Graph-Enhanced Large Language Models' or related concepts, feel free to ask!

--------------------------------------------------------------------------------
reader_user (to paper_reader):

MEMORIZE_ARTICLE: The following passage is extracted from an article titled 'GraphextQA: A Benchmark for Evaluating Graph-Enhanced Large Language Models [http://arxiv.org/pdf/2310.08487v1]': 

 relations from Wikidata. Notably, at least one of the answers is already covered in the paired graph in 97.8% of all samples, ensuring the relevance of the paired graph..

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
paper_reader (

FileNotFoundError: [Errno 2] No such file or directory: './teachability_db-3/uid_text_dict.pkl'